In [86]:
# Open the file
from collections import defaultdict, deque, Counter, namedtuple
from functools import reduce, cache, cmp_to_key
from heapq import heappush, heappop, heapify
from itertools import permutations, combinations, product
from math import gcd, sqrt, factorial, ceil, floor
from sortedcontainers import SortedDict, SortedList, SortedSet
#import PyRival
from typing import TypeAlias
from math import prod
import re
import numpy as np
import sys
import pyomo.environ as pymo
import pyomo.opt as pyopt
from .. import AOC_helpers as AOC
sys.setrecursionlimit(10000)

input_file = 'input.txt'
test_file = 'test.txt'

with open(input_file, 'r') as file:
    lines  = file.readlines()
    grid = [[x for x in line.strip()] for line in lines]


# ----------------------------------------------------------------------
directions = [(0,1), (-1, 0), (1,0), (0, -1)]
direction_dict =  {0:(0,1), 1:(-1, 0), 2:(1,0), 3:(0, -1)}


def construct_graph(grid):
    """Get adj list for the graph.
    nodes comprise of (r,c, dir)"""
    R = len(grid)
    C = len(grid[0])
    adj_list = defaultdict(list)
    costs = defaultdict(dict)
    start = None
    end = None
    for r in range(1, R - 1):
        for c in range(1, C - 1):
            if grid[r][c] != '#':
                if grid[r][c] == 'S':
                    start = r, c, 0
                elif grid[r][c] == 'E':
                    end = r, c
                for d in direction_dict:
                    for d2 in direction_dict:
                        # directions not same, can only rotate
                        if d != d2:
                            adj_list[(r, c, d)].append((r, c, d2))
                            if (r, c) == end:
                                # make rotation free at end so we can make 
                                # (end, 0) the end state. 
                                costs[(r, c, d)][(r, c, d2)] = 0
                            else: costs[(r, c, d)][(r,c, d2)] = 1000
                        # directions same, can only move.
                        else:
                            dr, dc = direction_dict[d2]
                            nr, nc = r + dr, c + dc 
                            if grid[nr][nc] != '#':
                                adj_list[(r, c, d)].append((nr, nc, d))
                                costs[(r, c, d)][(nr,nc, d)] = 1
                                
                
    return start, end, adj_list, costs

# given graph get shortest path from start to end.

start, end, adj_list, costs = construct_graph(grid)
nodes = adj_list.keys()
inf = float('inf')

def dijkstra(start, adj_list, costs):
    """Returns a dictionary where the keys are the nodes of our graph and the 
    values are the min dists of start to the given node """
    dists = defaultdict(lambda : inf)
    q = [(0, start)]
    while q:
        cur_cost, node = heappop(q)
        if dists[node] <= cur_cost:
            continue
        else:
            dists[node] = cur_cost     
            for neighbor in adj_list[node]:
                d = dists[node] + costs[node][neighbor]
                if d < dists[neighbor]:
                    heappush(q, (d, neighbor))
    return dists

# Compute min dist to end 
end = tuple(list(end) + [0])         
lowest_score = dijkstra(start, adj_list, costs)[end]
print(f'the lowest score is {lowest_score}')
            

ImportError: attempted relative import with no known parent package

In [82]:
# part 2: How many tiles touch a best path?
# find set of cells that are part of a best path.

# Idea: We know the score of the best path, just do like a bfs with keeping track of current path as we go. We may get a heuristic about how many turns we could use.

# cell is part of a best path if what? 
# Let L be the length of best path from start to cell and let R be the len of the best path of end to cell. L + R == B then the cell is part of a best path! 

# there are like 10k cells to look at. 

# for each cell, we can just do dijistra 1 and dijstra 2....
# this takes too  much time instead we call djisktra twice one time from start and the other from end and ask for every coordinate if sum of each is min dist 


reverse_dir = {0:3, 1:2, 2:1, 3:0} # node from end's perspective has direction reversed
dist_s = dijkstra(start, adj_list, costs)
dist_f = dijkstra(end, adj_list, costs)

best_path_nodes = set()

for i, node in enumerate(nodes):
    (r, c, d) = node
    if (r, c) in best_path_nodes:
        continue
    d1 = dist_s[node]
    rev_node = (r, c, reverse_dir[d])
    if d1 > lowest_score:
        continue
    d_2 = dist_f[rev_node]
    if d1 + d_2 == lowest_score:
        best_path_nodes.add((r, c))
    
nodes_on_best_path  = (len(best_path_nodes))
print(f'there are {nodes_on_best_path} nodes on a best path in the race')
    

there are 541 nodes on a best path in the race
